# Toronto Neighborhoods

## - [Exercise 1: Building up Postal Code DataFrame](#exe1)
## - [Excercise 2: Getting the Coordinates](#exe2)
## - [Excercise 3: Clustering Neighborhoods](#exe3)

 <a id=exe1></a>
# Exercise 1
## Building Postal Code DataFrame

In [1]:
import pandas as pd
import wikipedia as wp

Getting the table from Wikipedia.

In [2]:
html = wp.page("List_of_postal_codes_of_Canada:_M").html()
df = pd.read_html(html)[0] 

Dropping the rows with `Borough` == `'Not assigned'`

In [3]:
df.drop(df[df['Borough']=="Not assigned"].index, axis=0, inplace=True)
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


Replacing the `Neighbourhood` == `'Not Assined'` with the name of the `Borough`

In [4]:
df.rename(columns={"Postcode": "Postal Code"}, inplace=True)

In [5]:
df['Neighbourhood'] = df['Neighbourhood'].replace('Not assigned', df['Borough'])

In [6]:
df[df['Neighbourhood'] == 'Not assigned'].count() 

Postal Code      0
Borough          0
Neighbourhood    0
dtype: int64

Joining `Neighbourhood`columns with the same `Postcode`

In [7]:
df_clean = df.groupby(['Postal Code', 'Borough']).agg(lambda col: ', '.join(col))
df_clean.reset_index(inplace=True)
df_clean.head(20)

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


**The shape of the final DataFrame is:**

In [8]:
df_clean.shape

(103, 3)

<a id=exe2></a>
# Exercise 2 
## Getting neighborhood coordinates

In [9]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')

In [10]:
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the coordinates with the neighborhood names per matching `Postal Code`

In [11]:
df_coords = df_clean.merge(coordinates, on='Postal Code')

In [12]:
df_coords.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<a id=exe3></a>
# Exercise 3 

## Getting a map and clustering

In [13]:
import folium
from geopy.geocoders import Nominatim

In [14]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [15]:
df_toronto = df_coords[df_coords['Borough'].str.contains('Toronto')]

In [16]:
df_toronto.Neighbourhood.unique()

array(['The Beaches', 'The Danforth West, Riverdale',
       'The Beaches West, India Bazaar', 'Studio District',
       'Lawrence Park', 'Davisville North', 'North Toronto West',
       'Davisville', 'Moore Park, Summerhill East',
       'Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West',
       'Rosedale', 'Cabbagetown, St. James Town', 'Church and Wellesley',
       'Harbourfront', 'Ryerson, Garden District', 'St. James Town',
       'Berczy Park', 'Central Bay Street', 'Adelaide, King, Richmond',
       'Harbourfront East, Toronto Islands, Union Station',
       'Design Exchange, Toronto Dominion Centre',
       'Commerce Court, Victoria Hotel', 'Roselawn',
       'Forest Hill North, Forest Hill West',
       'The Annex, North Midtown, Yorkville',
       'Harbord, University of Toronto',
       'Chinatown, Grange Park, Kensington Market',
       'CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara',
       'Stn

Creating a map of toronto with the neighborhoods marked as dots. 

In [17]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, long, borough, neighborhood in zip(df_toronto['Latitude'], 
                                             df_toronto['Longitude'], 
                                             df_toronto['Borough'], 
                                             df_toronto['Neighbourhood']):
    label = '{},{}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.vector_layers.CircleMarker([lat, long], 
                        radius=5, 
                       popup=label,
                       color='blue',
                       fill=True, 
                       fill_color='3186cc', 
                       fill_opacity=0.7, 
                       parse_html=False).add_to(map_toronto)

map_toronto

**Getting the venue information from Foursquare**

In [18]:
CLIENT_ID = '...' # your Foursquare ID
CLIENT_SECRET = '...' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 100
radius = 500
import requests

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get all the close venues to each neighborhood. 

In [20]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                 latitudes=df_toronto['Latitude'],
                                 longitudes=df_toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

### Exploring the venues around the neighborhoods. 

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(1706, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [22]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",24,24,24,24,24,24
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",42,42,42,42,42,42
Central Bay Street,84,84,84,84,84,84
"Chinatown, Grange Park, Kensington Market",80,80,80,80,80,80
Christie,18,18,18,18,18,18


In [23]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 229 uniques categories.


**Encoding the categorical venue values in order to run the clustering algorithm**

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
toronto_onehot.shape

(1706, 229)

In [26]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.020000,...,0.010000,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.01
1,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.018182,0.000000,0.000000,0.000000,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.041667,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
6,Central Bay Street,0.011905,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.011905,...,0.000000,0.000000,0.000000,0.00000,0.00,0.011905,0.000000,0.000000,0.011905,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.050000,0.000000,0.050000,0.012500,0.00
8,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
9,Church and Wellesley,0.012346,0.012346,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012346,...,0.012346,0.012346,0.000000,0.00000,0.00,0.000000,0.000000,0.012346,0.000000,0.00


In [27]:
toronto_grouped.shape

(39, 229)

In [28]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0       Coffee Shop  0.07
1               Bar  0.04
2        Steakhouse  0.04
3              Café  0.04
4  Asian Restaurant  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2          Café  0.04
3    Steakhouse  0.04
4      Beer Bar  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.08
1            Café  0.08
2     Coffee Shop  0.08
3       Nightclub  0.08
4     Yoga Studio  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0      Recording Studio  0.06
1         Garden Center  0.06
2  Fast Food Restaurant  0.06
3            Skate Park  0.06
4                  Park  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                 venue  freq
0      Airport Service  0.19
1     Airport Terminal  0.12
2       Airpor

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

import numpy as np

In [56]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Steakhouse,Bar,Café,Cosmetics Shop,Breakfast Spot,Asian Restaurant,Burger Joint,Restaurant,Thai Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Steakhouse,Café,Cheese Shop,Beer Bar,Bakery,Breakfast Spot
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Nightclub,Yoga Studio,Bakery,Performing Arts Venue,Pet Store,Convenience Store,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eastern,Park,Burrito Place,Butcher,Fast Food Restaurant,Farmers Market,Auto Workshop,Recording Studio,Spa,Restaurant,Pizza Place
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Airport Lounge,Sculpture Garden,Rental Car Location,Harbor / Marina,Bar,Boat or Ferry,Coffee Shop,Airport Gate


## Clustering Neighborhoods

In [58]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 0, 2, 2, 2, 1, 2, 2])

In [59]:
df_toronto.rename(columns={"Neighbourhood": "Neighborhood"}, inplace=True)

C:\Users\sergi\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [60]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,9,Trail,Health Food Store,Pub,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Pub,Bakery,Dessert Shop
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,2,Park,Sushi Restaurant,Board Shop,Brewery,Italian Restaurant,Liquor Store,Burger Joint,Burrito Place,Ice Cream Shop,Fast Food Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Gastropub,Bakery,Brewery,Italian Restaurant,American Restaurant,Comfort Food Restaurant,Sandwich Place,Cheese Shop
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,6,Bus Line,Lake,Swim School,Park,Colombian Restaurant,Comfort Food Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [61]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.vector_layers.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 0

This cluster has only one neighborhood yet it seems to have all the necesities incluting a 5th most common venue of a market and park. It would be my ideal choice of living. Yet I will take a look more in depth at the other clusters. 

In [62]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
87,East Toronto,0,Park,Burrito Place,Butcher,Fast Food Restaurant,Farmers Market,Auto Workshop,Recording Studio,Spa,Restaurant,Pizza Place


### Cluster 1

This cluster looks good except that it does not have a market in its top venues. 

In [63]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,Downtown Toronto,1,Café,Bar,Bookstore,Sandwich Place,Japanese Restaurant,Restaurant,Bakery,Pub,Beer Bar,Beer Store
67,Downtown Toronto,1,Bar,Café,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Dumpling Restaurant,Mexican Restaurant,Donut Shop,Cocktail Bar
77,West Toronto,1,Bar,Restaurant,Asian Restaurant,Café,Vietnamese Restaurant,Men's Store,Coffee Shop,Pizza Place,Yoga Studio,Ice Cream Shop
82,West Toronto,1,Mexican Restaurant,Café,Thai Restaurant,Bar,Fried Chicken Joint,Bookstore,Diner,Italian Restaurant,Fast Food Restaurant,Cajun / Creole Restaurant
83,West Toronto,1,Gift Shop,Breakfast Spot,Coffee Shop,Movie Theater,Bookstore,Italian Restaurant,Dog Run,Bar,Restaurant,Dessert Shop


### Cluster 2

This cluster has the most neighborhoods and since most seem to be close to or in the downtown area, they do have a lot of options for venues. An analysis of prices for rent needs to be conducted to see if this cluster would have more expensive apartemnts than the other clusters. 

In [64]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Pub,Bakery,Dessert Shop
42,East Toronto,2,Park,Sushi Restaurant,Board Shop,Brewery,Italian Restaurant,Liquor Store,Burger Joint,Burrito Place,Ice Cream Shop,Fast Food Restaurant
43,East Toronto,2,Café,Coffee Shop,Gastropub,Bakery,Brewery,Italian Restaurant,American Restaurant,Comfort Food Restaurant,Sandwich Place,Cheese Shop
46,Central Toronto,2,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Salon / Barbershop,Restaurant,Rental Car Location,Café,Chinese Restaurant,Mexican Restaurant
47,Central Toronto,2,Dessert Shop,Sandwich Place,Gym,Coffee Shop,Sushi Restaurant,Pizza Place,Café,Italian Restaurant,Gourmet Shop,Indian Restaurant
49,Central Toronto,2,Pub,Coffee Shop,Liquor Store,Supermarket,Sports Bar,Fried Chicken Joint,Restaurant,Pizza Place,Sushi Restaurant,Light Rail Station
51,Downtown Toronto,2,Restaurant,Coffee Shop,Bakery,Café,Pizza Place,Italian Restaurant,Pub,Pharmacy,Playground,Snack Place
52,Downtown Toronto,2,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Men's Store,Pub,Gastropub,Fast Food Restaurant,Gym
53,Downtown Toronto,2,Coffee Shop,Pub,Park,Café,Bakery,Mexican Restaurant,Restaurant,Breakfast Spot,Event Space,Hotel
54,Downtown Toronto,2,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Cosmetics Shop,Diner,Bubble Tea Shop,Ice Cream Shop,Pizza Place,Bakery


### Cluster 3

This cluster with only one neighborhood, but not an abundancy of markets. 

In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3
                   ,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,3,Park,Playground,Tennis Court,Restaurant,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner


### Cluster 4

This cluster has one neighborhood with many buslines passing by. Maybe good for a commute. 

In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4
                   ,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Central Toronto,4,Bus Line,Jewelry Store,Trail,Sushi Restaurant,Women's Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


### Cluster 5

This cluster with one neighborhood, not my top choice based on venues. 

In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5
                   ,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,5,Health & Beauty Service,Pool,Garden,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


### Cluster 6

This cluster with one neighborhood, not my top choice based on venues. 

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6
                   ,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,6,Bus Line,Lake,Swim School,Park,Colombian Restaurant,Comfort Food Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Cluster 7

This cluster with one neighborhood, not my top choice based on venues.

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7
                   ,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,7,Park,Playground,Trail,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


### Cluster 8

This cluster with one neighborhood, not my top choice based on venues. Maybe hard to find an appartment since it is a hotel area mostly

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8
                   ,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Central Toronto,8,Hotel,Park,Gym,Breakfast Spot,Sandwich Place,Food & Drink Shop,Department Store,Women's Store,Donut Shop,Doner Restaurant


### Cluster 9

This cluster with one neighborhood, not my top choice based on venues.

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9
                   ,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,9,Trail,Health Food Store,Pub,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In my opinion after taking a look at the clusters I would go for Cluster 0 and see if I can find something there. If not then in cluster 2 seems to have a lot of choice. 